# **Model Architecture**
defining functions to create the generator and discriminator models for a Generative Adversarial Network (GAN). These models are used for image colorization.

In [ ]:
def d_block(x_in, fltr, strd, pad, bn, inorm):
    x = Conv2D(fltr, (4, 4),
               strides=strd,
               padding=pad,
               use_bias=False,
               kernel_initializer=init)(x_in)

    if bn:
        x = BatchNormalization()(x)
    if inorm:
        x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)
    return x

def u_block(x, skip, fltr, strd, pad, bn, inorm):
    x = Conv2DTranspose(fltr, (4, 4),
                        strides=strd,
                        padding=pad,
                        use_bias=False,
                        kernel_initializer=init)(x)

    if bn:
        x = BatchNormalization()(x)
    if inorm:
        x = InstanceNormalization()(x)
    x = ReLU()(x)
    conc_x = Concatenate()([x, skip])

    return conc_x


In [ ]:
def create_patchgan(image_shape):
    input_gen = Input(shape=image_shape)
    input_tar = Input(shape=image_shape)
    combined_inputs = Concatenate()([input_gen, input_tar])

    x64 = d_block(combined_inputs, 64, 2, 'same', False, False)
    x128 = d_block(x64, 128, 2, 'same', False, True)
    x256 = d_block(x128, 256, 2, 'same', True, False)

    padded_x256 = ZeroPadding2D()(x256)
    x512 = d_block(padded_x256, 512, 1, 'valid', True, False)

    padded_x512 = ZeroPadding2D()(x512)
    x1 = Conv2D(1, (4, 4), strides=1, padding='valid', activation='sigmoid', kernel_initializer=init)(padded_x512)

    model = Model(inputs=[input_gen, input_tar], outputs=x1)
    return model


In [ ]:
discriminatotr=create_patchgan((128, 128, 3))
discriminatotr.summary()

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 128, 128, 6)  0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [ ]:
def create_mod_unet():
    input_src = Input(shape=(128, 128, 3))

    x64 = d_block(input_src, 64, 2, 'same', False, False)
    x128 = d_block(x64, 128, 2, 'same', True, False)
    x256 = d_block(x128, 256, 2, 'same', True, False)
    x512 = d_block(x256, 512, 2, 'same', True, False)
    d512 = d_block(x512, 512, 2, 'same', True, False)
    e512 = d_block(d512, 512, 2, 'same', True, False)

    f512 = d_block(e512, 512, 2, 'same', True, False)

    u512 = u_block(f512, e512, 512, 2, 'same', True, False)
    u512 = u_block(u512, d512, 512, 2, 'same', True, False)
    u512 = u_block(u512, x512, 512, 2, 'same', True, False)
    u256 = u_block(u512, x256, 256, 2, 'same', True, False)
    u128 = u_block(u256, x128, 128, 2, 'same', True, False)
    u64 = u_block(u128, x64, 64, 2, 'same', False, True)

    generated_image = Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh', kernel_initializer=init)(u64)

    model = Model(inputs=input_src, outputs=generated_image)
    return model


In [ ]:
generator = create_mod_unet()
generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 64, 64, 64)   3072        ['input_3[0][0]']                
                                                                                                  
 leaky_re_lu_4 (LeakyReLU)      (None, 64, 64, 64)   0           ['conv2d_5[0][0]']               
                                                                                                  
 conv2d_6 (Conv2D)              (None, 32, 32, 128)  131072      ['leaky_re_lu_4[0][0]']    

# **Dataset Preparation**
 creating TensorFlow Dataset objects from the training and validation data.

In [ ]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

train = train.shuffle(buffer_size=400).batch(batch_size=16)
valid = valid.shuffle(buffer_size=400).batch(batch_size=16)

In [ ]:
gen_model = create_mod_unet()
dis_models = [create_patchgan((128, 128, 3)), create_patchgan((64, 64, 3)), create_patchgan((32, 32, 3))]

LAMBDA=100
optimizer_params = { "learning_rate": 0.0002,"beta_1": 0.5,"beta_2": 0.999}

gen_opt = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_0 = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_1 = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_2 = tf.keras.optimizers.Adam(**optimizer_params)

bce_loss = keras.losses.BinaryCrossentropy(from_logits=True)

# **Loss Functions**
defining custom loss functions for the generator and discriminator.

In [ ]:
def gen_loss(dis_gen_out, target_img, gen_img):
    adv_loss = bce_loss(tf.ones_like(dis_gen_out), dis_gen_out)
    l1_loss = tf.reduce_mean(tf.abs(tf.subtract(target_img, gen_img)))
    total_loss = adv_loss + (LAMBDA * l1_loss)
    return total_loss, adv_loss, l1_loss

def dis_loss(dis_gen_out, dis_target_out):
    gen_loss = bce_loss(tf.zeros_like(dis_gen_out), dis_gen_out)
    target_loss = bce_loss(tf.ones_like(dis_target_out), dis_target_out)
    total_dis_loss = gen_loss + target_loss
    return total_dis_loss